# Behavioral-Economic Analysis of Demand and Preference
### A Python Replication of Kirkman et al. (2022), *Learning and Motivation*

---

### Project Objective
This notebook provides a complete, reproducible Python workflow for the behavioral-economic analyses presented in the publication:

> Kirkman, C., Wan, H., & Hackenberg, T. D. (2022). A behavioral-economic analysis of demand and preference for social and food reinforcement in rats. *Learning and Motivation*, *77*, 101780. https://doi.org/10.1016/j.lmot.2021.101780

The study uses demand curve analysis to quantify the value of food and social reinforcers. It investigates how demand for each reinforcer changes as its own price increases (**own-price elasticity**) and how demand for one reinforcer changes as the price of the *other* reinforcer increases (**cross-price elasticity**). This allows for a quantitative assessment of whether the reinforcers act as substitutes, complements, or are independent.

**Note on Data:** This analysis requires the raw data file (`data with bl mean.csv`), assumed to be in a `/Data` subdirectory.

### Analytical Approach
The core of the analysis involves fitting specialized models from behavioral economics to consumption data using `scipy.optimize.curve_fit`:
1.  **Own-Price Demand**: The **Zero-Bounded Exponential (ZBEn) model** is fit to characterize how demand intensity ($Q_0$) and elasticity ($\alpha$) change with price.
2.  **Cross-Price Demand**: Both **linear** and **nonlinear exponential models** are fit to characterize how consumption of a constant-price reinforcer changes as the price of the alternative reinforcer varies.

### Analysis Workflow
1.  **Setup**: Load libraries, define helper functions, and load/prepare the dataset.
2.  **Phase 1 Analysis**: Model own-price demand for food and cross-price demand for social interaction.
3.  **Phase 2 Analysis**: Model own-price demand for social interaction and cross-price demand for food.
4.  **Phase 3 Analysis**: Model own-price demand for both reinforcers when their prices increase concurrently.
5.  **Phase 4 Analysis**: Model own-price demand for food in isolation and compare it to Phase 1.

In [22]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from lmfit import Model, Parameters
import warnings

# Suppress warnings from curve_fit, which can be verbose
warnings.filterwarnings("ignore")

# Set the display format for floating-point numbers to 3 decimal places
pd.options.display.float_format = '{:.3f}'.format

# --- Helper Functions (translated from R) ---
def lhs(x):
    """Inverse Hyperbolic Sine (log-like) transformation."""
    return np.log10(0.5 * x + np.sqrt(0.25 * (x**2) + 1))

# --- Load Data ---
behav_data = pd.read_csv("data with bl mean.csv")
# Add transformed consumption columns for modeling
behav_data['foodr_lhs'] = lhs(behav_data['foodr'])
behav_data['socr_lhs'] = lhs(behav_data['socr'])

In [23]:
# --- Environment Setup ---
#
# This cell installs the required Python packages for the analysis.
# Uncomment and run this cell only if you are setting up a new environment.

# import sys
# !{sys.executable} -m pip install pandas numpy scipy

In [24]:
# --- 1. SETUP: IMPORTS, FUNCTIONS, AND DATA PREPARATION ---

# --- 1.1 Load Libraries ---
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import warnings

# --- 1.2 Configure Environment ---
# Suppress warnings (e.g., from curve_fit) for a cleaner final report
warnings.filterwarnings("ignore")
# Set pandas display options for consistent formatting
pd.options.display.float_format = '{:.3f}'.format


# --- 1.3 Define Helper Function ---
def lhs(x):
    """
    Inverse Hyperbolic Sine (IHS) Transformation.

    Applies the IHS transformation, a log-like function that is defined at zero.
    This is required for the dependent variable in the ZBEn demand model.

    Args:
        x (float or np.ndarray): A numeric scalar or array.

    Returns:
        float or np.ndarray: The IHS-transformed value(s).
    """
    return np.log10(0.5 * x + np.sqrt(0.25 * (x**2) + 1))


# --- 1.4 Load and Prepare Data ---
# Load the pre-processed data containing session means.
raw_df = pd.read_csv("data with bl mean.csv")

# Prepare the data for modeling by applying the IHS transformation
# to the consumption rate columns.
analysis_df = raw_df.copy()
analysis_df['food_rate_ihs'] = lhs(analysis_df['foodr'])
analysis_df['social_rate_ihs'] = lhs(analysis_df['socr'])

## 2. Demand Curve Analysis

This section replicates the model fitting in the order presented in the Results section of the published article. Each subsection corresponds to an experimental phase.

### Phase 1: Food Price Varied, Social Price Constant (FR 1)

In this phase, the price of food was systematically increased across sessions, while the price of social interaction was held constant at a low price (FR 1). This allows us to perform two key analyses:

1.  **Own-Price Demand for Food**: We fit the **Zero-Bounded Exponential (ZBEn) model** to the food consumption data. This analysis quantifies how demand for food changes as its own price increases. We expect to see consumption decrease as price increases.
2.  **Cross-Price Demand for Social Interaction**: We model how consumption of the constant-price social reinforcer changes as the price of food increases. An increase in social consumption as food becomes more expensive would indicate a **substitutable relationship** between the two reinforcers.

In [25]:
# --- 2.1 Import lmfit and Define Model Functions ---
from lmfit import Model, Parameters

def zbe_model_func(price, alpha, q0):
    """
    Zero-Bounded Exponential (ZBEn) demand model.
    Models IHS-transformed consumption as a function of price.
    """
    q0 = max(q0, 1e-9) # Prevent division by zero
    ihs_q0 = lhs(q0)
    return ihs_q0 * np.exp((-alpha / ihs_q0) * q0 * price)

def cross_price_exp_func(price, beta, q_alone, i):
    """
    Exponential cross-price elasticity model from Hursh (2014).
    A negative 'i' parameter indicates substitutability.
    """
    return np.log10(q_alone) + i * np.exp(-beta * price)

# --- 2.2 Phase 1 Analysis: Food Price Varied ---

# Filter data for Phase 1
phase1_df = analysis_df[analysis_df['cond'] == 1].copy()

def fit_lmfit_model(group, model_obj, x_col, y_col, param_config):
    """Helper function to apply an lmfit model to a group."""
    params = Parameters()
    for name, config in param_config.items():
        params.add(name, **config)
        
    result = model_obj.fit(group[y_col], params, price=group[x_col])
    return pd.Series(result.best_values)

# --- Own-Price Demand for Food (ZBEn Model) ---
print("--- Phase 1: Own-Price Food Demand Parameters (ZBEn Model) ---")
zbe_model_obj = Model(zbe_model_func, independent_vars=['price'])
zbe_params_config = {
    'alpha': {'value': 0.0001, 'min': 0},
    'q0': {'value': 50, 'min': 0}
}

own_price_food_p1 = phase1_df.groupby('subj').apply(
    fit_lmfit_model,
    model_obj=zbe_model_obj,
    x_col='foodfr',
    y_col='food_rate_ihs',
    param_config=zbe_params_config
).reset_index()
display(own_price_food_p1)

# --- Cross-Price Demand for Social Interaction (Exponential Model) ---
print("\n--- Phase 1: Cross-Price Social Demand Parameters (Exponential Model) ---")
cross_price_model_obj = Model(cross_price_exp_func, independent_vars=['price'])
cross_price_params_config = {
    'beta': {'value': 0.01, 'min': 0},
    'q_alone': {'value': 100, 'min': 0},
    'i': {'value': 1}
}

cross_price_social_p1 = phase1_df.groupby('subj').apply(
    fit_lmfit_model,
    model_obj=cross_price_model_obj,
    x_col='foodfr',
    y_col='socr',
    param_config=cross_price_params_config
).reset_index()
display(cross_price_social_p1)

--- Phase 1: Own-Price Food Demand Parameters (ZBEn Model) ---


,subj,alpha,q0
0,1,0.000,177.745
1,2,0.000,260.448
2,3,0.000,393.347
3,4,0.000,217.576



--- Phase 1: Cross-Price Social Demand Parameters (Exponential Model) ---


,subj,beta,q_alone,i
0,1,0.000,231422.032,31.369
1,2,319044590961.900,135935571189553077440617644032.000,-63.011
2,3,248.734,912010838418402909757689809298942069607235584.000,249.541
3,4,2808.828,41900790867313343393169408.000,-94060.842


### Phase 2: Social Price Varied, Food Price Constant (FR 1)

In this phase, the experimental conditions were reversed: the price of social interaction was systematically increased across sessions, while the price of food was held constant at a low price (FR 1). This allows us to test the other side of the reinforcer interaction:

1.  **Own-Price Demand for Social Interaction**: We fit the **ZBEn model** to the social interaction data to quantify how demand for social access changes as its own price increases.
2.  **Cross-Price Demand for Food**: We model how consumption of the constant-price food reinforcer changes as the price of social interaction increases. If the two reinforcers are independent in this context, we would expect to see little to no change in food consumption.

In [26]:
# --- 2.3 Phase 2 Analysis: Social Price Varied ---

# Filter data for Phase 2
phase2_df = analysis_df[analysis_df['cond'] == 2].copy()

# --- Own-Price Demand for Social Interaction (ZBEn Model) ---
print("--- Phase 2: Own-Price Social Demand Parameters (ZBEn Model) ---")
own_price_social_p2 = phase2_df.groupby('subj').apply(
    fit_lmfit_model,
    model_obj=zbe_model_obj,
    x_col='socfr',
    y_col='social_rate_ihs',
    param_config=zbe_params_config
).reset_index()
display(own_price_social_p2)

# --- Cross-Price Demand for Food (Exponential Model) ---
print("\n--- Phase 2: Cross-Price Food Demand Parameters (Exponential Model) ---")
# Note: Initial values are adjusted based on the original code for better convergence.
cross_price_food_params_config = {
    'beta': {'value': 0.1},
    'q_alone': {'value': 10000},
    'i': {'value': 100}
}

cross_price_food_p2 = phase2_df.groupby('subj').apply(
    fit_lmfit_model,
    model_obj=cross_price_model_obj,
    x_col='socfr',
    y_col='foodr',
    param_config=cross_price_food_params_config
).reset_index()
display(cross_price_food_p2)

--- Phase 2: Own-Price Social Demand Parameters (ZBEn Model) ---


,subj,alpha,q0
0,1,0.004,36.600
1,2,0.003,67.848
2,3,0.011,15.308
3,4,0.004,48.438



--- Phase 2: Cross-Price Food Demand Parameters (Exponential Model) ---


,subj,beta,q_alone,i
0,1,-0.074,48903725672879118247209911232216662836371682503...,15.598
1,2,-0.000,51667840.729,196.441
2,3,0.437,15727811085874138695788032088721050081728706810...,55.178
3,4,-0.100,13989955163120218782379878852813264451657508803...,22.842


### Phase 3: Concurrent Price Increase

In this phase, the prices of both food and social reinforcers were increased together across sessions. This allows for a direct, within-subject comparison of the **own-price demand** for each reinforcer when both are becoming progressively more expensive.

We again fit the **ZBEn model** to the consumption data for each reinforcer separately. This analysis helps determine which reinforcer retains its value more effectively under escalating costs. Note that only two subjects (Rats 4 and 6) completed this phase of the experiment.

In [27]:
# --- 2.4 Phase 3 Analysis: Concurrent Price Increase ---

# Filter data for Phase 3
phase3_df = analysis_df[analysis_df['cond'] == 3].copy()

# To avoid repeating code, we'll transform the data from wide to long format.
# This allows us to fit models for both reinforcer types in a single pipeline.
phase3_long_df = phase3_df.melt(
    id_vars=['subj', 'foodfr', 'socfr'],
    value_vars=['food_rate_ihs', 'social_rate_ihs'],
    var_name='reinforcer_type',
    value_name='consumption_ihs'
)

# Create a 'price' column corresponding to each reinforcer type
phase3_long_df['price'] = np.where(
    phase3_long_df['reinforcer_type'] == 'food_rate_ihs',
    phase3_long_df['foodfr'],
    phase3_long_df['socfr']
)

# --- Fit Own-Price Demand Models for Both Reinforcers ---
print("--- Phase 3: Own-Price Demand Parameters (ZBEn Model) ---")
own_price_p3 = phase3_long_df.groupby(['subj', 'reinforcer_type']).apply(
    fit_lmfit_model,
    model_obj=zbe_model_obj,
    x_col='price',
    y_col='consumption_ihs',
    param_config=zbe_params_config
).reset_index()

display(own_price_p3)

--- Phase 3: Own-Price Demand Parameters (ZBEn Model) ---


,subj,reinforcer_type,alpha,q0
0,2,food_rate_ihs,0.000,159.777
1,2,social_rate_ihs,0.001,64.046
2,3,food_rate_ihs,0.000,195.924
3,3,social_rate_ihs,0.005,55.452


### Phase 4: Food Demand in Isolation (No Social Alternative)

In this final phase, the demand for food was assessed when it was the only available reinforcer. The price of food was systematically increased across sessions, just as in Phase 1, but with the social interaction option removed entirely.

This design allows for a direct comparison of the **own-price demand for food** in two different economic contexts: with a potential substitute available (Phase 1) versus without one (Phase 4). By comparing the demand parameters ($\alpha$ and $Q_0$) between these phases, we can quantify how the presence of a social alternative impacts the value of food. Note that only two subjects (Rats 4 and 6) completed this phase.

In [28]:
# --- 2.5 Phase 4 Analysis: Food Price Varied in Isolation ---

# Filter data for Phase 4
phase4_df = analysis_df[analysis_df['cond'] == 4].copy()

# --- Fit Own-Price Demand Model for Food ---
print("--- Phase 4: Own-Price Food Demand Parameters (ZBEn Model, No Social Alternative) ---")
own_price_food_p4 = phase4_df.groupby('subj').apply(
    fit_lmfit_model,
    model_obj=zbe_model_obj,
    x_col='foodfr',
    y_col='food_rate_ihs',
    param_config=zbe_params_config
).reset_index()

display(own_price_food_p4)

--- Phase 4: Own-Price Food Demand Parameters (ZBEn Model, No Social Alternative) ---


,subj,alpha,q0
0,2,0.000,273.148
1,3,0.000,193.248
